In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
# Get all the class links
url = "https://catalog.upenn.edu/courses/"

response = requests.get(url)
header = "https://catalog.upenn.edu"
course_links = []

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    course_div = soup.find_all('div', class_='az_sitemap')

    course_uls = course_div[0].find_all('ul', class_=False)

    # Get and store all the course links
    for course_ul in course_uls:
        course_lis = course_ul.find_all('li', class_=False)
        for course_li in course_lis:
            a_elements = course_li.find_all('a')
            href_value = a_elements[0].get('href')
            course_links.append(header + href_value)
else:
    print("Failed to retrieve the page. Status code:", response.status_code)

In [13]:
def get_course_names(url):
    response = requests.get(url)
    course_numbers, course_names = [], []

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        course_listings = soup.find_all('div', class_='courseblock')

        for course in course_listings:
            p_tags = course.find_all('p')
            
            course_info = p_tags[0].text
            course_number = course_info.split('  ')[0].replace('\xa0', ' ')
            course_name = course_info.split('  ')[1]
            course_numbers.append(course_number)
            course_names.append(course_name)

    else:
        print("Failed to retrieve the page. Status code:", response.status_code)
    return course_numbers, course_names


In [21]:
number_name = {}

for course_link in course_links:
    course_numbers, course_names = get_course_names(course_link)

    for i in range(len(course_numbers)):
        number_name[course_numbers[i]] = course_names[i]

In [22]:
# Store it to a json file
with open("course_name_map.json", 'w') as f:
    json.dump(number_name, f)

# Number description relationship

In [23]:
# Connect course number to description
def get_course_descriptions(url):
    response = requests.get(url)
    course_numbers, course_descriptions = [], []

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        course_listings = soup.find_all('div', class_='courseblock')

        for course in course_listings:
            p_tags = course.find_all('p')
            
            course_info = p_tags[0].text
            course_number = course_info.split('  ')[0].replace('\xa0', ' ')
            course_description = p_tags[1].text.split('\n')[0].replace('\xa0', ' ')
            course_numbers.append(course_number)
            course_descriptions.append(course_description)

    else:
        print("Failed to retrieve the page. Status code:", response.status_code)
    return course_numbers, course_descriptions

In [34]:
number_description = {}

for course_link in course_links:
    course_numbers, course_descriptions = get_course_names(course_link)

    for i in range(len(course_numbers)):
        number_description[course_numbers[i]] = course_descriptions[i]

In [32]:
# Store it to a json file
with open("course_description_map.json", 'w') as f:
    json.dump(number_description, f)